# **Analyse de sentiments sur les critiques spectateurs sur Allociné**
**Projet Python pour la Data science - 2A ENSAE**

Zakaria BOULLIAIRE, Massyle DENDENE, Brian RAMESH

# Introduction

L'idée de ce projet est de prédire ("mettre la variable à prédire"), à partir d'une analyse de sentiment faite sur les critiques données pas les spectateurs (et non la presse), sur le site Allociné. 
Nous allons donc consituer une base de données de film, en scrappant le site Allociné et en utilisant l'API de The Movie Database (TMDB) pour compléter les données manquantes.

### Imports nécessaires

In [3]:
%%time
# BeutifulSoup pour le scrapping
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import requests

# Classique python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint

from warnings import warn
from time import sleep

import seaborn as sns

from datetime import datetime
from dateutil import parser 

CPU times: user 2.3 s, sys: 4.29 s, total: 6.6 s
Wall time: 925 ms


# Collecte des données

### Scrapping sur Allociné

Tout d'abord, nous allons restreindre notre base de données au film américain des années 2010 à 2021. Cela constiuerait une base de données de 11 822 films.

Les données qui peuvent nous être utiles sont les suivantes : Titre original du film, identifiant du film sur Allociné (qui nous sera utile pour récupérer les critiques plus tard), la note des spectateurs et celle de la presse, le nombre de critiques presse, le nombre de critiques spectateurs et le nombre de votes pour la notes (dans une seule et même variable, qu'on spérera lors du nettoyage), la date de sortie du film, le budget et le Box Office US. 
Nous avions également commencer à scrapper le N°Visa du film, qui est unique pour chaque film, en vue d'utiliser cette donnée pour la collecte sur TMDB, mais il y avait énormément de film pour lesquels cette donnée manquait. Nous avons décider de ne pas l'utiliser.

L'idée de la fonction ci dessous est de scrapper les données dont on a besoin à partir de l'url et du nombre de page.

In [4]:
%%time
def scrapping_film_allocine(base_url, nb_page):

    # Liste ou on stock nos données
    data = []
    
    # Variable de comptage pour voir l'évolution du scrapping, et détecter les eventuelles erreurs
    i=0

    # Boucle sur les pages
    for page in range(1, nb_page+1): 
        url_page_ac = f"{base_url}{page}"

        response_page_ac = requests.get(url_page_ac)

        if response_page_ac.status_code == 200:
            bs_page_ac = bs(response_page_ac.text, "html.parser")
            films_page_ac = bs_page_ac.findAll("li", attrs={'class': "mdl"})

            for film_allocine in films_page_ac:
                i+=1
                try:
                    
                    # Id du film sur allo cine
                    meta_title_link = film_allocine.find('a', class_='meta-title-link')

                    if meta_title_link:
                        href1 = meta_title_link.get('href')
                        film_id = href1.split('=')[-1].split('.')[0]
                    else:
                        film_id = None


                    # Scrapping de la page fiche info du film qu'on obtient grace à l'id trouvé ci dessus
                    url_fiche_film = f'https://www.allocine.fr/film/fichefilm_gen_cfilm={film_id}.html'
                    response_fiche_film = requests.get(url_fiche_film)
                    bs_fiche_film = bs(response_fiche_film.text, "html.parser")

                    # Titre
                    titre_allocine = meta_title_link.text

                    span_titre_original = bs_fiche_film.find('span', class_='light', string='Titre original ')
                    titre_original = span_titre_original.find_next_sibling(string=True).strip() if span_titre_original else titre_allocine


                    # Notes spectateurs/presse, nombre critiques presse/spectateurs
                    bloc_notes = bs_fiche_film.findAll('span', class_='stareval-note')
                    list_notes = [notes.get_text(strip=True) for notes in bloc_notes]

                    if len(list_notes)==0:
                        note_presse = None
                        note_spectateur = None
                        
                    else:
                        index_delimiteur = list_notes.index('--')
                        new_liste_notes = list_notes[:index_delimiteur]

                        if len(new_liste_notes)==2:
                            note_presse = new_liste_notes[0]
                            note_spectateur = new_liste_notes[1]
                        
                        elif len(new_liste_notes) > 0 and len(new_liste_notes) <= 1:
                            note_spectateur = new_liste_notes[0]
                            note_presse = None

                    bloc_critiques = bs_fiche_film.find_all('span', class_='stareval-review')

                    if len(bloc_critiques)==2:
                        critiques_element_presse = bloc_critiques[0].text
                        critiques_element_spec = bloc_critiques[1].text
                    elif len(bloc_critiques) > 0 and len(bloc_critiques) <= 1:
                        critiques_element_spec = bloc_critiques[0].text
                        critiques_element_presse = None
                    else:
                        critiques_element_presse = None
                        critiques_element_spec = None


                    #Date, durée, budget
                    date_film_element = film_allocine.find('span', class_='date')
                    date_film = date_film_element.text if date_film_element else None

                    duree_film_element = bs_fiche_film.find('span', class_='spacer')
                    duree_film = duree_film_element.next_sibling.strip() if duree_film_element else None

                    budget_element = bs_fiche_film.find('span', class_='what light', string='Budget')
                    budget_film = budget_element.find_next('span').string if budget_element else None

                    #visa_element = bs_fiche_film.find('span', class_='what light', string='N° de Visa')
                    #visa_film = visa_element.find_next('span').string if visa_element else None

                    # Box office
                    url_box_office = f'https://www.allocine.fr/film/fichefilm-{film_id}/box-office/'
                    response_box_office = requests.get(url_box_office)
                    bs_box_office = bs(response_box_office.text, "html.parser")

                    cumul = bs_box_office.findAll('td', {'data-heading': 'Cumul'})
                    list_cumul = [cum.get_text(strip=True) for cum in cumul]

                    box_office_film = list_cumul[-1] if list_cumul else 'None'

                    data.append([titre_original, note_presse, note_spectateur, critiques_element_presse, critiques_element_spec, film_id, box_office_film, budget_film,
                                date_film, duree_film])
                    
                    df_data = pd.DataFrame(data, columns=["Titre original", "Note press", "Notes spectateur", "Critiques presse", "Critiques spectateurs", 'id allocine',
                                          'Box office', 'Budget', 'date', 'duree'])


                    print(i)
                except Exception as e:
                    print(f"Une erreur s'est produite pour le film {i} : {e}")

    return df_data

CPU times: user 4 µs, sys: 5 µs, total: 9 µs
Wall time: 13.8 µs


In [ ]:
# Urls qu'on veut scrapper
base_url_2010_2020, nb_page_2010_2020 = 'https://www.allocine.fr/films/pays-5002/decennie-2010/?page=', 646
base_url_2020, nb_page_2020 = 'https://www.allocine.fr/films/pays-5002/decennie-2020/annee-2020/?page', 66
base_url_2021, nb_page_2021 = 'https://www.allocine.fr/films/pays-5002/decennie-2020/annee-2021/?page' 77


# Application de la foncrion scrapping_film_allocine
df_data_2010_2020 = scrapping_film_allocine(base_url_2010_2020, nb_page_2010_2020)
df_data_2020 = scrapping_film_allocine(base_url_2020, nb_page_2020)
df_data_2021 = scrapping_film_allocine(base_url_2021, nb_page_2021)

# Créer un DataFrame avec les données collectées
dfs=[]
dfs.append(df_data_2010_2020)
dfs.append(df_data_2020)
dfs.append(df_data_2021)

df_film_ac = pd.concat(dfs, ignore_index=True)

### Nettoyage de la base obtenue

In [4]:
df_movies_allocine= pd.read_csv("df_film_ac.csv")

In [5]:
df_movies_allocine

,Titre original,Note press,Notes spectateur,Critiques presse,Critiques spectateurs,id allocine,Box office,Budget,date,duree
0,Shutter Island,"3,8","4,4",31 critiques,80471 notes dont 4605 critiques,132039,127 770 000,80 000 000 $,24 février 2010,2h 17min
1,Inception,"4,1","4,5",24 critiques,110095 notes dont 7212 critiques,143692,290 948 208,160 000 000 $,21 juillet 2010,2h 28min
2,Harry Potter and the Deathly Hallows - Part 1,"3,2","4,0",20 critiques,52676 notes dont 2887 critiques,126693,291 377 000,150 000 000 $,24 novembre 2010,2h 26min
3,Prince of Persia: The Sands of Time,"2,6","3,1",22 critiques,26730 notes dont 2133 critiques,126678,89 981 000,200 000 000 $,26 mai 2010,2h 06min
4,The Book of Eli,"2,4","3,3",20 critiques,10503 notes dont 1144 critiques,128955,92 524 000,80 000 000 $,20 janvier 2010,1h 49min
...,...,...,...,...,...,...,...,...,...,...
11816,Amarillo,NaN,NaN,NaN,NaN,294872,NaN,-,NaN,NaN
11817,Chastise,NaN,NaN,NaN,NaN,295695,NaN,-,NaN,NaN
11818,Tapawingo,NaN,NaN,NaN,NaN,296176,NaN,-,NaN,NaN
11819,Mr. Birthday,NaN,NaN,NaN,NaN,298844,NaN,-,NaN,NaN


Comme on peut voir, la base a besoin d'être nettoyé. Nous allons donc supprimer tous les films qui n'ont pas de note spectateurs

In [12]:
print(df_movies_allocine.dtypes)

Titre original           object
Note press               object
Notes spectateur         object
Critiques presse         object
Critiques spectateurs    object
id allocine               int64
Box office               object
Budget                   object
date                     object
duree                    object
dtype: object


In [17]:
# On modifie les types numériques 

df_movies_allocine['Box office'] = df_movies_allocine['Box office'].str.replace(' ', '').astype(float)
df_movies_allocine['Note press'] = df_movies_allocine['Note press'].str.replace(',', '.').astype(float)
df_movies_allocine['Notes spectateur'] = df_movies_allocine['Notes spectateur'].replace("--", np.nan)
df_movies_allocine['Notes spectateur'] = df_movies_allocine['Notes spectateur'].str.replace(',', '.').astype(float)

In [18]:
print(df_movies_allocine.dtypes)

Titre original            object
Note press               float64
Notes spectateur         float64
Critiques presse          object
Critiques spectateurs     object
id allocine                int64
Box office               float64
Budget                    object
date                      object
duree                     object
dtype: object


In [6]:
# Fonction pour convertir le mois en anglais
def french_to_english_month(month_french):
    months_mapping = {
        'janvier': 'January',
        'février': 'February',
        'mars': 'March',
        'avril': 'April',
        'mai': 'May',
        'juin': 'June',
        'juillet': 'July',
        'août': 'August',
        'septembre': 'September',
        'octobre': 'October',
        'novembre': 'November',
        'décembre': 'December'
    }
    return months_mapping.get(month_french.lower(), month_french)


# Fonction pour convertir la durée en minutes
def convert_duration(duration_str):
    if isinstance(duration_str, str):
        # Supprimer les espaces et diviser la chaîne en parties
        parts = duration_str.replace(' ', '').split('h')

        # Vérifier la présence des heures et des minutes
        if len(parts) == 2:
            hours = int(parts[0])
            minutes = 0 if 'min' not in parts[1] else int(parts[1].replace('min', ''))
            
            # Calculer la durée totale en minutes
            total_minutes = hours * 60 + minutes
            
            return int(total_minutes)
    
    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return float('nan')



#Extraire le nombre de critique de la presse 

def extract_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Utiliser isdigit() pour extraire uniquement les chiffres
        return np.nan if not critiques_str.split()[0].isdigit() else int(critiques_str.split()[0])
    else:
        return np.nan



# Fonction pour extraire le nombre de notes et le nombre de critiques
def extract_notes_and_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Trouver les nombres dans la chaîne
        numbers = [int(word) for word in critiques_str.split() if word.isdigit()]

        # Extraire le nombre de notes et de critiques en fonction de la longueur de la liste "numbers"
        if len(numbers) == 1:
            return numbers[0], np.nan
        elif len(numbers) == 2:
            return numbers[0], numbers[1]

    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return np.nan, np.nan

In [7]:
## Appliacation de nos fonctions

# Remplacer les chaînes "nan" par des valeurs NaN
df_movies_allocine['date'] = df_movies_allocine['date'].replace('nan', np.nan)

# Appliquer la fonction pour convertir le mois en anglais
df_movies_allocine['date'] = df_movies_allocine['date'].apply(lambda x: ' '.join([french_to_english_month(word) for word in str(x).split()]) if pd.notna(x) else np.nan)

# Utiliser dateutil.parser.parse pour convertir les dates en objets datetime
df_movies_allocine['date'] = df_movies_allocine['date'].apply(lambda x: parser.parse(x, dayfirst=True) if isinstance(x, str) else x)


# Appliquer la fonction de conversion pour la durée
df_movies_allocine['duree_minutes'] = df_movies_allocine['duree'].apply(convert_duration)

# Appliquer la fonction extract_notes_and_critiques_count
df_movies_allocine[['Nombre_de_notes_spectateurs', 'Nombre_de_critiques_spectateurs']] = df_movies_allocine['Critiques spectateurs'].apply(extract_notes_and_critiques_count).apply(pd.Series)

# Appliquer la fonction extract_critiques_count
df_movies_allocine['Nombre_de_critiques_presse'] = df_movies_allocine['Critiques presse'].apply(extract_critiques_count)

In [8]:
df_movies_allocine

,Titre original,Note press,Notes spectateur,Critiques presse,Critiques spectateurs,id allocine,Box office,Budget,date,duree,duree_minutes,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs,Nombre_de_critiques_presse
0,Shutter Island,"3,8","4,4",31 critiques,80471 notes dont 4605 critiques,132039,127 770 000,80 000 000 $,2010-02-24,2h 17min,137.0,80471.0,4605.0,31.0
1,Inception,"4,1","4,5",24 critiques,110095 notes dont 7212 critiques,143692,290 948 208,160 000 000 $,2010-07-21,2h 28min,148.0,110095.0,7212.0,24.0
2,Harry Potter and the Deathly Hallows - Part 1,"3,2","4,0",20 critiques,52676 notes dont 2887 critiques,126693,291 377 000,150 000 000 $,2010-11-24,2h 26min,146.0,52676.0,2887.0,20.0
3,Prince of Persia: The Sands of Time,"2,6","3,1",22 critiques,26730 notes dont 2133 critiques,126678,89 981 000,200 000 000 $,2010-05-26,2h 06min,126.0,26730.0,2133.0,22.0
4,The Book of Eli,"2,4","3,3",20 critiques,10503 notes dont 1144 critiques,128955,92 524 000,80 000 000 $,2010-01-20,1h 49min,109.0,10503.0,1144.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11816,Amarillo,NaN,NaN,NaN,NaN,294872,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN
11817,Chastise,NaN,NaN,NaN,NaN,295695,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN
11818,Tapawingo,NaN,NaN,NaN,NaN,296176,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN
11819,Mr. Birthday,NaN,NaN,NaN,NaN,298844,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN


In [9]:
# Retrait des colonnes inutiles 
colonnes_a_retirer = ["Critiques presse","Critiques spectateurs","duree","Budget"]
df_movies_allocine = df_movies_allocine.drop(colonnes_a_retirer,axis=1)

In [ ]:
df_movies_allocine

In [11]:
# Retrait de tous les films sans critique spectateurs
# On garde que les films avec au moins plus de 5 commentaires

df_film_avec_critiques = df_movies_allocine.dropna(subset=['Nombre_de_critiques_spectateurs'])
df_film_avec_critiques = df_film_avec_critiques[df_film_avec_critiques['Nombre_de_critiques_spectateurs'] >= 5]

In [12]:
df_film_avec_critiques

,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs,Nombre_de_critiques_presse
0,Shutter Island,"3,8","4,4",132039,127 770 000,2010-02-24,137.0,80471.0,4605.0,31.0
1,Inception,"4,1","4,5",143692,290 948 208,2010-07-21,148.0,110095.0,7212.0,24.0
2,Harry Potter and the Deathly Hallows - Part 1,"3,2","4,0",126693,291 377 000,2010-11-24,146.0,52676.0,2887.0,20.0
3,Prince of Persia: The Sands of Time,"2,6","3,1",126678,89 981 000,2010-05-26,126.0,26730.0,2133.0,22.0
4,The Book of Eli,"2,4","3,3",128955,92 524 000,2010-01-20,109.0,10503.0,1144.0,20.0
...,...,...,...,...,...,...,...,...,...,...
11101,The Gateway,NaN,"2,3",273584,NaN,2022-03-16,91.0,40.0,6.0,NaN
11102,"Batman: The Long Halloween, Part Two",NaN,"3,7",292641,NaN,2021-08-04,84.0,114.0,7.0,NaN
11104,Mortal Kombat Legends: Battle of the Realms,NaN,"3,3",294101,NaN,2021-09-15,80.0,50.0,6.0,NaN
11117,Black As Night,NaN,"1,6",287632,NaN,2021-10-01,87.0,53.0,6.0,NaN
